In [10]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model

In [71]:
EMBEDDING_FILE = 'data/glove.6B.300d.txt'
df = pd.read_csv('data/train.csv').fillna(" ")


In [72]:
X.head()

61457     "\n\n STOP VANDALIZING \n\nPlease stop your di...
28251     "\nSaid from someone who had some very offensi...
42900     Media:Example.oggInsert non-formatted text her...
75000     "\n\n Good to see that a lot of this content i...
126592                     Hight towers company for towers.
Name: comment_text, dtype: object

In [76]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Replace NA values
df['comment_text'].fillna(' ')

# Separate input features and target
y = df.loc[:, "toxic"]
X = df.loc[:,'comment_text']


# split into train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)




# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
non_toxic = X[X["toxic"]==0]
toxic = X[X["toxic"]==1]

# upsample minority
toxic_upsampled = resample(toxic,
                          replace=True, # sample with replacement
                          n_samples=len(non_toxic), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
X_upsampled = pd.concat([non_toxic, toxic_upsampled])


# AGAIN: Separate input features and target
y = X_upsampled.loc[:, "toxic"]
X = X_upsampled.loc[:,'comment_text']


'''
    QUESTION: Followed: https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18
    for upsampling where it states to split before upsampling to avoid overfitting the data. After they split the data, they
    resample the minority class, and then concatenate the results to a single Dataframe. In their model examples,
    it seems as though they are training their models on the entirity of the dataset and resuing data that their model has
    already seen for the test results, which should result in overfitting, right?
    
    
    So once I have the balanced upsampled dataset, I've resplit the dataset in training and testing

'''

# split into train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)






num_non_toxic = X_upsampled.loc[X_upsampled["toxic"] == 0]
num_toxic = X_upsampled.loc[X_upsampled["toxic"] == 1]


print("Class Distribution across 2 classes:")
print(len(num_non_toxic),len(num_toxic))


max_features=100000
maxlen=150
embed_size=300


Class Distribution across 2 classes:
108285 108285


In [74]:
# Vectorize text + Prepare GloVe Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)


In [28]:
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [34]:
# NOTE: NOT CURRENTLY WORKING
# I thought I'd keep this is in to visualize how a Sequential approach might work

# from keras.layers.core import Merge
from keras.layers import Concatenate
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import Multiply

input_dim = 32
hidden = 32
step = 1

#The LSTM  model -  output_shape = (batch, step, hidden)
model1 = Sequential()
model1.add(LSTM(input_dim=input_dim, output_dim=hidden, input_length=step, return_sequences=True))

#The weight model  - actual output shape  = (batch, step)
# after reshape : output_shape = (batch, step,  hidden)
model2 = Sequential()
model2.add(Dense(input_dim=input_dim, output_dim=step))
model2.add(Activation('softmax')) # Learn a probability distribution over each  step.
#Reshape to match LSTM's output shape, so that we can do element-wise multiplication.
model2.add(RepeatVector(hidden))
model2.add(Permute((2, 1)))


#The final model which gives the weighted sum:
model = Sequential()
model.add(Multiply([model1, model2]))  # Multiply each element with corresponding weight a[i][j][k] * b[i][j]
model.add(TimeDistributedMerge('sum')) # Sum the weighted elements.

model.compile(loss='binary_crossentropy', optimizer='adam')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(1, 32), units=32)`
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=32, units=1)`


TypeError: __init__() takes 1 positional argument but 2 were given

In [47]:
from keras.layers import Input, Flatten, Activation, multiply, TimeDistributed, RepeatVector, Permute, Lambda
from keras import backend as K

units = 32
max_length = 150
vocab_size = embedding_matrix.shape[0]
embedding_size = embedding_matrix.shape[1]


_input = Input(shape=[max_length], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=max_length,
        trainable=False,
        weights=[embedding_matrix]
    )(_input)

# Bidirectional(LSTM(128, return_sequences=True))
# activations = LSTM(units, return_sequences=True)(embedded)
# Bidirectional(LSTM(64, return_sequences=True))
activations = Bidirectional(LSTM(32, return_sequences=True))(embedded)


# compute importance for each step
attention = TimeDistributed(Dense(1, activation='tanh'))(activations) 
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
# adjusting vector shapes for multiplication
attention = RepeatVector(64)(attention)
attention = Permute([2, 1])(attention)

# apply the attention

sent_representation = multiply([activations, attention])

sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation) 

probabilities = Dense(1, activation='sigmoid')(sent_representation)

model = Model(input=_input, output=probabilities)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [75]:
# Prediction
batch_size = 32

model.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=1)


Epoch 1/1
162427/162427 [==============================] - 960s 6ms/step - loss: 0.3941 - acc: 0.8168


In [58]:
predictions = model.predict(X_test, verbose=1)

153164/153164 [==============================] - 294s 2ms/step


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

accuracy_score(y_test, predictions)


f1_score(y_test, predictions)

    
recall_score(y_test, predictions)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(epochs=[10,20,30], batch_size=[10, 20, 30], optimizer=['SGD', 'Adam'])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)


In [ ]:
print(grid_result.best_params_)
print(grid_result.best_score_)

### TODO:

Dec 18
 
* Email Milad and CC Farhad about GPU usage
* Try Downsampling if GPU usage is not possible
* Compare sole BiLSTM w/ model with Attn
* Compare Eval metrics between lstm and cnn
* Check Farhad email for parameters: loss function, activation of the output Dense layer
* Make sure outputing probabilities
* Generating confusion matrix for training and validation data
* Try different n-grams (filter-size) for the CNN model 
--> Obtaining specficity and sensitvity
* Check out Perspective API CNN implementation






-----------------------------------------

* Upsample the minority class

* Eval Metrics:
Select One
Sensitivity(Recall) and specificity, Percision, F1 Score
Calculate both for training and test

AUC 

Focus on model eval for LSTM and then compare to CNN performance

Hyperparamter tuning for LSTM and CNN models

NOTE: there should be balance between sensitivity, specificity
